## Overview ##

PubChem is a site run by the NIH which hosts raw data associated with chemical experiments; here we analyze the data hosted at PubChem for assay 1030, which looks for inhibitors of the protein encoding gene ALDH1A1. You can access the page for this assay [here](https://pubchem.ncbi.nlm.nih.gov/bioassay/1030)

## Results ##

We use the SMILES string, a common representation for a molecule amongst chemists, to begin the featurization process. Because the length of this string varies, it is normalized in the form of a Morgan Fingerprint; these are then used to train various DNN-based classifiers in this notebook

In [1]:
# Exploratory data analysis and visualization

In [2]:
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
import sys

import warnings
warnings.filterwarnings('ignore')

global_random_state = 42

np.random.seed(global_random_state)


active_pct = 0.073125471
inactive_pct = 1 - active_pct


import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fh = logging.FileHandler('log_dnn.txt')
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
logger.addHandler(fh)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

In [3]:
import keras
print(keras.backend.backend())

Using TensorFlow backend.


tensorflow


In [4]:
# What about a deep neural network?
# Sample code from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import pickle

k_fold_splits = 2
global_random_state = 42

with open('data.classification.undersampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)

class_weights = compute_class_weight('balanced', np.unique(y), y)

    
def create_model() :
    model = Sequential()
    model.add(Dense(12, input_dim=2048, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model

skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)


for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=1)
    classifier.fit(X_train,y_train,class_weight=class_weights)
    y_pred = classifier.predict(X_test)
    logger.info(classification_report(y_test, y_pred))


Epoch 1/10
16110/16110 [==============================] - 4s - loss: 0.6531 - acc: 0.6145     
Epoch 2/10
16110/16110 [==============================] - 0s - loss: 0.5821 - acc: 0.6999     
Epoch 3/10
16110/16110 [==============================] - 0s - loss: 0.5444 - acc: 0.7316     
Epoch 4/10
16110/16110 [==============================] - 0s - loss: 0.5151 - acc: 0.7520     
Epoch 5/10
16110/16110 [==============================] - 0s - loss: 0.4842 - acc: 0.7714     
Epoch 6/10
16110/16110 [==============================] - 0s - loss: 0.4561 - acc: 0.7910     
Epoch 7/10
16110/16110 [==============================] - 0s - loss: 0.4269 - acc: 0.8094     
Epoch 8/10
16110/16110 [==============================] - 0s - loss: 0.3972 - acc: 0.8263     
Epoch 9/10
16110/16110 [==============================] - 0s - loss: 0.3642 - acc: 0.8453     
Epoch 10/10
14400/16112 [=========================>....] - ETA: 0s2017-09-24 14:23:31,958 - INFO -              precision    recall  f1-score   s

### 

In [5]:
# Let's try using DNN on the full non-sampled dataset, but with class_weight set and larger network. It didn't work

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, f1_score

import pickle

k_fold_splits = 2
global_random_state = 42

logger.info("Trying a larger network")

def create_model() :
    model = Sequential()
    model.add(Dense(1024, input_dim=2048, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model


skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=1)
    classifier.fit(X_train,y_train,class_weight=class_weights)
    y_pred = classifier.predict(X_test)
    logger.info(classification_report(y_test, y_pred))

2017-09-24 14:23:40,921 - INFO - Trying a larger network
Epoch 1/10
16110/16110 [==============================] - 2s - loss: 0.6333 - acc: 0.6379     
Epoch 2/10
16110/16110 [==============================] - 1s - loss: 0.4921 - acc: 0.7657     
Epoch 3/10
16110/16110 [==============================] - 1s - loss: 0.2921 - acc: 0.8765     
Epoch 4/10
16110/16110 [==============================] - 1s - loss: 0.1443 - acc: 0.9438     
Epoch 5/10
16110/16110 [==============================] - 1s - loss: 0.0834 - acc: 0.9712     
Epoch 6/10
16110/16110 [==============================] - 1s - loss: 0.0478 - acc: 0.9834     
Epoch 7/10
16110/16110 [==============================] - 1s - loss: 0.0501 - acc: 0.9843     
Epoch 8/10
16110/16110 [==============================] - 1s - loss: 0.0277 - acc: 0.9909     
Epoch 9/10
16110/16110 [==============================] - 1s - loss: 0.0217 - acc: 0.9937     
Epoch 10/10
15800/16112 [============================>.] - ETA: 0s2017-09-24 14:23:57,07